In [ ]:
!pip install mglearn
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import mglearn

#분류와 회귀


- 분류
    - 미리 정의된, 가능성 있는 여러 **클래스 레이블** 중 하나를 예측하는 것
    - (ex)1장에서 붓꽃을 세 품종 중 하나로 분류하는 것
    - 분류는 딱 두 개의 클래스로 분류하는 **이진 분류**와 셋 이상의 클래스로 분류하는 **다중 분류**로 나뉨

- 회귀
    - 연속적인 숫자, 실수를 예측하는 것
    - (ex)어떤 사람의 교육 수준, 나이, 주거지를 바탕으로 연간 소득을 예측하는 것
    - 예상 출력 값 사이에 연속성이 있다면 회귀 문제


# 일반화, 과대적합, 과소적합

- 일반화
    - 모델이 처음 보는 데이터에 대해 정확하게 예측할 수 있으면 이를 훈련 세트에서 테스트 세트로 **일반화**되었다고 함
    - 모델을 만들 때는 가능한 한 정확하게 일반화되도록 해야함
- 과대적합
    - 가진 정보를 모두 사용해서 너무 복잡한 모델을 만드는 것
    - 훈련 세트의 각 샘플에 너무 가깝게 맞춰져서 새로운 데이터에 일반화되기 어려울 때 일어남
- 과소적합
    - 모델이 너무 간단하여 데이터의 면면과 다양성을 잡아내지 못하고 훈련 세트에도 잘 맞지 않는 것
- 데이터셋에 다양한 데이터 포인트가 많을수록 과대적합 없이 더 복잡한 모델을 만들 수 있다. <br>그러나 같은 데이터 포인트를 중복하거나 매우 비슷한 데이터를 모으는 것이 도움이 되지 않는다.



# 지도 학습 알고리즘


- 예제에 사용할 데이터셋
1. forge 데이터셋

In [ ]:
X, y = mglearn.datasets.make_forge()
mglearn.discrete_scatter(X[:, 0], X[:, 1], y)
plt.legend(["class 0", "class 1"], loc = 4)
plt.xlabel("First trait")
plt.ylabel("Second trait")


2. wave 데이터셋

In [ ]:
X, y = mglearn.datasets.make_wave(n_samples=40)
plt.plot(X, y, 'o')
plt.ylim(-3, 3)
plt.xlabel("trait")
plt.ylabel("target")

3.Wisconsin Breast Cancer Datasets

In [ ]:
from sklearn.datasets import load_breast_cancer
cancer = load_breast_cancer()
print("cancer.keys():\n", cancer.keys())

- scikit-learn에 포함된 데이터셋은 실제 데이터와 데이터셋 관련 정보를 담고 있는 Bunch 객체에 저장되어 있다. Bunch 객체는 파이썬 딕셔너리와 비슷하지만 점 표기법이 사용가능하다.(bunch['key'] -> bunch.key)

In [ ]:
print("유방암 데이터의 형태:", cancer.data.shape)

In [ ]:
print("클래스별 샘플 개수:\n",
      {n: v for n, v in zip(cancer.target_names, np.bincount(cancer.target))})

In [ ]:
print("특성 이름:\n", cancer.feature_names)

#KNN(k-nearest neighbors) Algorithm



- knn 알고리즘은 새로운 데이터 포인트에 대해 예측할 때 알고리즘이 훈련 데이터셋에서 가장 가까운 데이터 포인트, 즉 '최근접 이웃'을 찾는다.
---


- 가장 간단한 k-nn 알고리즘은 가장 가까운 훈련 데이터 포인트하나를 최근접 이웃으로 찾아 예측에 사용한다.

In [ ]:
mglearn.plots.plot_knn_classification(n_neighbors=1)

- 이 그림에는 데이터 포인트 3개를 새로 추가했고 각 데이터 포인트에서 가장 가까운 훈련 데이터 포인트 하나를 연결했다.<br>**1-최근접 이웃 알고리즘**의 예측은 이 데이터 포인트의 레이블이 된다.

In [ ]:
mglearn.plots.plot_knn_classification(n_neighbors=3)


- 가장 가까운 이웃 하나가 아니라 임의의 k개를 선택할 수 있는데<br> 이 그림에선 테스트 포인트 하나에 대해 클래스 0에 속한 이웃이 몇 개인지, 그리고 클래스 1에 속한 이웃이 몇 개인지를 센 후 이웃이 더 많은 클래스를 레이블로 지정한다.

<h2>scikit-learn을 사용한 knn algorithm</h2>


In [ ]:
from sklearn.model_selection import train_test_split
X, y = mglearn.datasets.make_forge()

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(n_neighbors=3)

In [ ]:
clf.fit(X_train, y_train)


In [ ]:
print("테스트 세트 예측:", clf.predict(X_test))

In [ ]:
print("테스트 세트 정확도: {:.2f}".format(clf.score(X_test, y_test)))

<h2>KNeighborsClassifier분석</h2>

- 앞에서 보았던 2차원 데이터셋을 xy평면도에 그려보고 각 데이터 포인트가 속한 클래스에 따라 평면에 색을 칠하면<br> 클래스 0과 클래스 1로 지정한 영역으로 나뉘는  **결정 경계**를 볼 수 있다.

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(10, 3))

for n_neighbors, ax in zip([1, 3, 9], axes):
    clf = KNeighborsClassifier(n_neighbors=n_neighbors).fit(X, y)
    mglearn.plots.plot_2d_separator(clf, X, fill=True, eps=0.5, ax=ax, alpha=.4)
    mglearn.discrete_scatter(X[:, 0], X[:, 1], y, ax=ax)
    ax.set_title("{} Neighbors".format(n_neighbors))
    ax.set_xlabel("trait 0")
    ax.set_ylabel("trait 1")
axes[0].legend(loc=3)

- 위 그림을 보면 이웃을 하나 선택했을 떄는 결정 경계가 훈련 데이터에 가깝게 따라가고 이웃의 수를 늘릴수록 결정 경계는 더 부드러워진다.
- 부드러운 경계는 더 단순한 모델을 의미하는데 다시 말해 이웃을 적게 사용하면 모델의 복잡도가 높아지고 많이 사용하면 복잡도는 낮아진다.

In [ ]:
from sklearn.datasets import load_breast_cancer
cancer = load_breast_cancer()
X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target
, stratify=cancer.target, random_state=66)
training_accuracy = []
test_accuracy = []
neighbors_settings = range(1, 11)
for n_neighbors in neighbors_settings:
    clf = KNeighborsClassifier(n_neighbors=n_neighbors)
    clf.fit(X_train, y_train)
    training_accuracy.append(clf.score(X_train, y_train))
    test_accuracy.append(clf.score(X_test, y_test))

plt.plot(neighbors_settings, training_accuracy, label="Training accuracy")
plt.plot(neighbors_settings, test_accuracy, label="Test accuracy")
plt.ylabel("정확도")
plt.xlabel("n_neighbors")
plt.legend()

- n_neighbors 수에 따른 훈련 세트와 테스트 세트 정확도
    - 이웃의 수가 적을수록 모델이 복잡해짐
    - 이웃의 수가 늘어나면 모델은 단순해지고 훈련 데이터의 정확도는 줄어듬
    

In [ ]:
mglearn.plots.plot_knn_regression(n_neighbors=1)

In [ ]:
print(np.arange(8).reshape(2, 4))